In [ ]:

import tensorflow as tf
import numpy as np
import tensorflow_model_optimization as tfmot

import tempfile

input_shape = [20]
x_train = np.random.randn(1, 20).astype(np.float32)
y_train = tf.keras.utils.to_categorical(np.random.randn(1), num_classes=20)

def setup_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(20, input_shape=input_shape),
      tf.keras.layers.Flatten()
  ])
  return model

def setup_pretrained_weights():
  model = setup_model()
  model.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
  )
  model.fit(x_train, y_train)
  _, pretrained_weights = tempfile.mkstemp('.tf')
  model.save_weights(pretrained_weights)

  return pretrained_weights

def get_gzipped_model_size(model):
  # Returns size of gzipped model, in bytes.
  import os
  import zipfile

  _, keras_file = tempfile.mkstemp('.h5')
  model.save(keras_file, include_optimizer=False)

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(keras_file)

  return os.path.getsize(zipped_file)

# setup
setup_model()
pretrained_weights = setup_pretrained_weights()
print(pretrained_weights)

In [ ]:
'''Define model'''
# 修剪整个模型（顺序和功能）
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended.
model_for_pruning_all = tfmot.sparsity.keras.prune_low_magnitude(base_model)
model_for_pruning_all.summary()

In [ ]:
# 修剪一些层（顺序层和功能层）
# 与从头开始训练相比，通常最好通过修剪来进行微调。
# 尝试修剪后几层而不是前几层。
# 避免修剪关键层（例如注意机制）。

# Helper function uses `prune_low_magnitude` to make only the 
# Dense layers train with pruning. 仅修剪Dense图层。
def apply_pruning_to_dense(layer):
  if isinstance(layer, tf.keras.layers.Dense):
    return tfmot.sparsity.keras.prune_low_magnitude(layer)
  return layer

# Use `tf.keras.models.clone_model` to apply `apply_pruning_to_dense` 
# to the layers of the model.
model_for_pruning_some = tf.keras.models.clone_model(
    base_model,
    clone_function=apply_pruning_to_dense,
)
model_for_pruning_some.summary()

In [ ]:
'''Train model'''
# Define the model.
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)

log_dir = tempfile.mkdtemp()
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),   
    # Log sparsity and other metrics in Tensorboard.
    tfmot.sparsity.keras.PruningSummaries(log_dir=log_dir)
    # 可选项作为超参数
]

model_for_pruning.compile(
      loss=tf.keras.losses.categorical_crossentropy,
      optimizer='adam',
      metrics=['accuracy']
)

model_for_pruning.fit(
    x_train,
    y_train,
    callbacks=callbacks,
    epochs=2,
)
model_for_pruning.summary()

_, keras_model_file = tempfile.mkstemp('.h5')
print(keras_model_file)
# Checkpoint: saving the optimizer is necessary (include_optimizer=True is the default).
model_for_pruning.save(keras_model_file, include_optimizer=True)

In [ ]:
# Deserialize model. 反序列化模型
with tfmot.sparsity.keras.prune_scope():
  loaded_model = tf.keras.models.load_model(keras_model_file)

loaded_model.summary()

In [ ]:
# Define the model.
base_model = setup_model()
base_model.load_weights(pretrained_weights) # optional but recommended for model accuracy
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model)
# strip_pruning应用标准压缩算法（例如通过gzip）
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning) 
print("final model")
model_for_export.summary()

print("\n")
print("Size of gzipped pruned model without stripping: %.2f bytes" % (get_gzipped_model_size(model_for_pruning)))
print("Size of gzipped pruned model with stripping: %.2f bytes" % (get_gzipped_model_size(model_for_export)))

In [ ]:
# 增大块大小将降低目标模型精度可达到的峰值稀疏性
base_model = setup_model()
base_model.load_weights(pretrained_weights) 
# For using intrinsics on a CPU with 128-bit registers, together with 8-bit
# quantized weights, a 1x16 block size is nice because the block perfectly
# fits into the register.
pruning_params = {'block_size': [1, 16]}
model_for_pruning = tfmot.sparsity.keras.prune_low_magnitude(base_model, **pruning_params)

model_for_pruning.summary()